In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import random
import os

In [2]:
# Generate a new set of 1,500 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
len(lngs)

1500

In [3]:
#Get the nearest city using the citipy module
cities = [citipy.nearest_city(lat_lng[0],lat_lng[1]) for lat_lng in lat_lngs ]
unique_cities = []
for city in set(cities):
    unique_cities.append(city)

In [4]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

<ul>
    <li>Perform an API call with the OpenWeatherMap.</li>
    <li>Retrieve the following information from the API call:
        <ul>
            <li>Latitude and longitude</li>
            <li>Maximum temperature</li>
            <li>Percent humidity</li>
            <li>Percent cloudiness</li>
            <li>Wind speed</li>
            <li>Weather description (e.g., clouds, fog, light rain, clear sky)</li>
            <li>Using a try-except block, in the try block if it is raining, get the amount of rainfall in inches for the last three hours. In the except block handle the KeyError if there is no rainfall and add 0 inches for the rainfall amount.</li>
            <li>Using a try-except block, in the try block if it is snowing, get the amount of snowfall in inches for the last three hours. In the except block handle the KeyError if there is no snowfall and add 0 inches for the snowfall amount.</li>
        </ul>
    </li>
</ul>
    


In [6]:
# Retrieve a list of city_weather json objects
# In order not to keep sending requests to the openweathermap service, this code segment captures the json responses
# and uses them to create a list of city_weather json objects to be processed in later segments. 
# The code checks each response for the ["coord"] key. If an exception is thrown, the next request is submitted. If 
# there is no exception, the json response is appended to the city_weather_ls list.

city_weather_ls = []

for city in unique_cities:
    city_name = city.city_name
    city_url = url + "&q=" + city_name
    
    # Make a 'Get' request for the city weather.
    city_weather = requests.get(city_url).json()
    
    try:
        city_coord = city_weather["coord"]  
        city_weather_ls.append(city_weather)
    except (KeyError):
        # city not found
        continue
    
    

In [37]:
# Output file
outfolder = "weather_data"
outfile = "WeatherPy_challenge.csv"

city_weather_list = []


for city_weather in city_weather_ls:
    
    # Capture the weather parameters
    try:
        city_name = city_weather["name"]
        city_country = city_weather["sys"]["country"]
        city_latitude = city_weather["coord"]["lat"]
        city_longitude = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_cloudiness = city_weather["clouds"]["all"]
        city_wind_speed = city_weather["wind"]["speed"]
        city_weather_desc = city_weather["weather"][0]["description"]

    except (KeyError) as ke:
        # City not found. Move to next city.
        # print(f"KeyError: {ke}")
        continue
    
    try:
        city_rain_fall = city_weather["rain"]["3h"] # millimetres
        city_rain_fall = city_rain_fall / 25.4  # convert to inches
    except (KeyError):
        city_rain_fall = 0
    
    try:
        city_snow_fall = city_weather["snow"]["3h"] # millimetres
        city_snow_fall = city_snow_fall / 25.4  # convert to inches
    except (KeyError):
        city_snow_fall = 0
    
    city_weather_list.append({"Name":city_name,"Country":city_country,"Lat":city_latitude,"Lng":city_longitude,\
          "Max Temp":city_max_temp,"Humidity":city_humidity,"Cloudiness":city_cloudiness,\
          "Wind Speed":city_wind_speed,"Current Description":city_weather_desc,\
          "Rain (inches)":city_rain_fall,"Snow (inches)": city_snow_fall
         })
 
city_weather_df = pd.DataFrame(city_weather_list)
#city_weather_df["Lat"]
city_weather_df["Lat"] = city_weather_df["Lat"].map("{:.2f}".format)
city_weather_df["Lng"] = city_weather_df["Lng"].map("{:.2f}".format)
city_weather_df["Max Temp"] = city_weather_df["Max Temp"].map("{:.2f}".format)
city_weather_df["Rain (inches)"] = city_weather_df["Rain (inches)"].map("{:.1f}".format)
city_weather_df["Snow (inches)"] = city_weather_df["Snow (inches)"].map("{:.1f}".format)

city_weather_df.to_csv(os.path.join(outfolder,outfile))
city_weather_df

,Name,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Evensk,RU,61.95,159.23,36.68,80,75,3.98,broken clouds,0.0,0.0
1,Barabinsk,RU,55.36,78.36,64.36,60,82,3.06,broken clouds,0.0,0.0
2,Lárdos,GR,36.09,28.02,73.40,49,0,16.11,clear sky,0.0,0.0
3,Kawalu,ID,-7.38,108.21,74.48,91,29,0.72,scattered clouds,0.0,0.0
4,Porto Velho,BR,-8.76,-63.90,68.00,68,75,5.82,broken clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
584,Kaduqli,SD,11.02,29.72,97.83,26,66,2.89,broken clouds,0.0,0.0
585,Bilma,NE,18.69,12.92,108.01,8,99,12.46,overcast clouds,0.0,0.0
586,Grand-Lahou,CI,5.14,-5.02,83.43,76,19,12.03,few clouds,0.0,0.0
587,Bukachacha,RU,52.98,116.92,47.25,54,70,4.03,broken clouds,0.0,0.0


Q. How many cities have recorded rainfall or snow?

In [50]:
city_weather_df = pd.read_csv(os.path.join(outfolder,outfile))
city_weather_df[(city_weather_df["Rain (inches)"] > 0) | (city_weather_df["Snow (inches)"] > 0)].count()

Unnamed: 0             0
Name                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Rain (inches)          0
Snow (inches)          0
dtype: int64